In [2]:
import pandas as pd
import warnings
import numpy as np
import sklearn
from keras.layers import Dense, Activation, Input
from keras.models import Sequential



In [3]:
data_train=pd.read_csv("MergetrainDataSVD.csv") #read train file
data_test=pd.read_csv("MergetestDataSVD.csv")  #read  test fil
print data_train.shape
print data_test.shape

(15626, 511)
(4249, 512)


In [4]:
data_train.head(2)

,Unnamed: 0,0,entry,0.entry,entry.entry,2,3,4,5,6,...,498,499,colm,col10_new_10,col2_new_2,col3_new_3,col4_new_4,col6_new_6,col7_new_7,col_new_1
0,0,MGINKKDGQPQYAPSHLGTKPVKYKRNKGEKFHDKSNGHPIVMQTKGE,['A7Z570'],0.013624,0.001468,0.002604,0.000007,0.001220,-0.011797,-0.002597,...,-0.000152,-0.000167,colm1,0,0,0,0,0,0,1
1,1,MSSVDVLLTVGKLDASLALLTTQDHHVIEFPTVLLPENVKAGSIIK...,['Q12114'],0.005838,-0.003215,-0.000098,-0.004258,0.002207,-0.011858,0.001375,...,-0.000328,-0.000109,colm1,0,0,0,0,0,0,1


In [5]:
data_train1=data_train.drop(["colm"],axis=1)  #remove colm from train data 
data_test1=data_test.drop(["colm","GOTerm"],axis=1) #remove Goterm and colm
X_data_train=data_train1.values
X_data_test=data_test1.values
X_data_train=X_data_train[:,3:]#remove 1st 3 useless feture
X_data_test=X_data_test[:,3:]

In [6]:
X_data_train.shape

(15626, 507)

In [7]:
X_train=X_data_train[:,:500] # X train final train data
Y_train=X_data_train[:,500:] # Y multlevel data
X_test=X_data_test[:,:500]   #test data
Y_test=X_data_test[:,500:]   #multilevel y test

In [8]:
print X_train.shape
print Y_train.shape
print X_test.shape,Y_test.shape

(15626, 500)
(15626, 7)
(4249, 500) (4249, 7)


In [9]:
Y_test[:,1].sum()

389

In [10]:
X_test=sklearn.preprocessing.normalize(X_test, axis=1, copy=True, return_norm=False) #normalize x test
X_train=sklearn.preprocessing.normalize(X_train, axis=1, copy=True, return_norm=False)#normalize x train

In [11]:
#encoder model is used to dimensional reduction from 500 to 125
Encoder = Sequential()
Encoder.add( Dense(300, input_shape = (500,) ) ) #input shape is equal to no of feature in X_train
Encoder.add(Activation('sigmoid'))               
Encoder.add(Dense(200))
Encoder.add(Activation("relu"))
Encoder.add(Dense(125))
Encoder.add( Activation('sigmoid') )
Encoder.summary()
# 2 hidden layer 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               150300    
_________________________________________________________________
activation_1 (Activation)    (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               60200     
_________________________________________________________________
activation_2 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 125)               25125     
_________________________________________________________________
activation_3 (Activation)    (None, 125)               0         
Total params: 235,625
Trainable params: 235,625
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Decoder model is used to decoding from 125 to 500 feture
Decoder = Sequential()
Decoder.add( Dense(200, input_shape = (125,)) ) #input shape is output of encoder
Decoder.add(Activation("sigmoid"))
Decoder.add(Dense(300))
Decoder.add(Activation("relu"))
Decoder.add(Dense(500))
Decoder.add( Activation('sigmoid') )
Decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 200)               25200     
_________________________________________________________________
activation_4 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 300)               60300     
_________________________________________________________________
activation_5 (Activation)    (None, 300)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 500)               150500    
_________________________________________________________________
activation_6 (Activation)    (None, 500)               0         
Total params: 236,000
Trainable params: 236,000
Non-trainable params: 0
_________________________________________________________________


In [13]:
AutoEncoder = Sequential([Encoder, Decoder]) #autoencoder is used for encoding and  decoding both
AutoEncoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 125)               235625    
_________________________________________________________________
sequential_2 (Sequential)    (None, 500)               236000    
Total params: 471,625
Trainable params: 471,625
Non-trainable params: 0
_________________________________________________________________


In [14]:
AutoEncoder.compile(optimizer='rmsprop', loss='mse',metrics=["accuracy"]) 

In [17]:
from keras.callbacks import ModelCheckpoint 
filepath="Autoencodermodel.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor="loss" ,verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
#used to save the model which has less loss 

In [25]:
AutoEncoder.fit(X_train,X_train,batch_size=64, epochs=20,callbacks=callbacks_list,shuffle=True)

Epoch 1/20
15626/15626 [==============================] - 2s 134us/step - loss: 0.0019 - acc: 0.1507

Epoch 00001: loss improved from 0.00194 to 0.00194, saving model to Autoencodermodel.hdf5
Epoch 2/20
15626/15626 [==============================] - 2s 139us/step - loss: 0.0019 - acc: 0.1508

Epoch 00002: loss improved from 0.00194 to 0.00194, saving model to Autoencodermodel.hdf5
Epoch 3/20
15626/15626 [==============================] - 2s 138us/step - loss: 0.0019 - acc: 0.1516

Epoch 00003: loss improved from 0.00194 to 0.00194, saving model to Autoencodermodel.hdf5
Epoch 4/20
15626/15626 [==============================] - 2s 143us/step - loss: 0.0019 - acc: 0.1511

Epoch 00004: loss improved from 0.00194 to 0.00194, saving model to Autoencodermodel.hdf5
Epoch 5/20
15626/15626 [==============================] - 2s 140us/step - loss: 0.0019 - acc: 0.1517

Epoch 00005: loss improved from 0.00194 to 0.00194, saving model to Autoencodermodel.hdf5
Epoch 6/20
15626/15626 [================

In [166]:
AutoEncoder.save("after_AutoEnoder.hdf5") #save autoencoder model with less loss
Encoder.save("Enoder_1500.hdf5") #save encoder model

In [20]:
#AutoEncoder.evaluate(X_train[1000].reshape(1,500),X_train[1000].reshape(1,500))

In [26]:
X_train_autoencoder=Encoder.predict(X_train) #reduce train dimension by encoder model
X_test_autoencoder=Encoder.predict(X_test)#reduce test dimension by encoder model

In [31]:
Y_test=Y_test.astype("float32") #change dtype obj to float
Y_train=Y_train.astype("float32") #change dtype obj to float

In [32]:
print X_test.shape
print Y_train.dtype

(4249, 500)
float32


In [27]:
from sklearn.metrics import classification_report #for classification report precision,recall and f1-score

from skmultilearn.problem_transform import BinaryRelevance
#BinaryRelevanceis  the simplest technique, which basically treats each label as a separate single class classification problem.


from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [45]:
classifier = BinaryRelevance(svm.SVC(kernel='rbf', class_weight="balanced"))
    #BinaryRelevance with svm and kernal is rbf
classifier.fit(X_train_autoencoder, Y_train)
   # predict
Y_predicted_SVM_BinaryRelevance= classifier.predict(X_test_autoencoder) #

print "acc",accuracy_score(Y_test,Y_predicted_SVM_BinaryRelevance)
print classification_report(Y_test,Y_predicted_SVM_BinaryRelevance)


In [36]:
#BinaryRelevance with rbf
classifier = BinaryRelevance(RandomForestClassifier(class_weight="balanced"))

classifier.fit(X_train_autoencoder, Y_train)
# predict
Y_predicted_rbf_BinaryRelevance= classifier.predict(X_test_autoencoder)

print "acc",accuracy_score(Y_test,Y_predicted_rbf_BinaryRelevance)
print classification_report(Y_test,Y_predicted_rbf_BinaryRelevance)


acc 0.4290421275594257
             precision    recall  f1-score   support

          0       0.67      0.21      0.32       282
          1       0.88      0.33      0.48       389
          2       0.64      0.59      0.62      1567
          3       0.94      0.87      0.90       557
          4       0.71      0.35      0.47       989
          5       0.54      0.08      0.13       465
          6       0.76      0.24      0.36       129

avg / total       0.71      0.46      0.53      4378



In [37]:
# using classifier chains and rbf
from skmultilearn.problem_transform import ClassifierChain

# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
classifier = ClassifierChain(RandomForestClassifier(class_weight="balanced"))

# train
classifier.fit(X_train_autoencoder, Y_train)

# predict
Y_predicted_cChain = classifier.predict(X_test_autoencoder)

acc=accuracy_score(Y_test,predictions)
print acc
print classification_report(Y_test,Y_predicted_cChain)

0.5342433513767946
             precision    recall  f1-score   support

          0       0.65      0.16      0.26       282
          1       0.87      0.34      0.49       389
          2       0.62      0.58      0.60      1567
          3       0.93      0.87      0.90       557
          4       0.56      0.53      0.55       989
          5       0.33      0.26      0.29       465
          6       0.52      0.57      0.54       129

avg / total       0.64      0.53      0.56      4378



In [38]:
from skmultilearn.adapt import MLkNN
#multi-label version of kNN is represented by MLkNN

classifier = MLkNN(k=7) #no of cluster 

# train
classifier.fit(X_train_autoencoder, Y_train)

# predict
predictions = classifier.predict(X_test_autoencoder)

print accuracy_score(Y_test,predictions)
print classification_report(Y_test,predictions)

0.48105436573311366


In [40]:
from scipy import sparse
#sparse_y_test=sparse.csr_matrix(Y_test) to change numpy array to sparse matrix
from scipy.sparse import csr_matrix
predictions=csr_matrix(predictions, dtype=np.int8).toarray() #convert sparse matrix to an array

In [42]:
Y_predicted_df=predictions

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test.argmax(axis=1), Y_predicted_df.argmax(axis=1))

In [43]:
listMetrics =[]
listMetrics.append( ["TP","FP","TN","FN","Precision","Recall"])
totalPrecision = 0.0
totalRecall = 0.0
for i in range(Y_test.shape[0]):
    listM = []
    truePositive = 0.0
    trueNegative = 0.0
    falsePositive = 0.0
    falseNegative = 0.0
    precision = 0.0
    recall = 0.0
    for j in range(Y_test.shape[1]):
        if Y_predicted_df[i,j] == 1:
            if Y_test[i,j] == 1:
                truePositive = truePositive + 1
            else:    
                falsePositive = falsePositive + 1
        else:
            if Y_test[i,j] == 1:
                falseNegative = falseNegative + 1
            else:
                trueNegative = trueNegative + 1
    try:
        precision = truePositive/(truePositive + falsePositive)
    except ZeroDivisionError:
        precision=0
    totalPrecision = totalPrecision + precision
    try:
        recall = truePositive/(truePositive + falseNegative)
    except ZeroDivisionError:
        recall=0
    totalRecall = totalRecall + recall
    
    listM.append(truePositive)
    listM.append(falsePositive)
    listM.append(trueNegative)
    listM.append(falseNegative)
    listM.append(precision)
    listM.append(recall)
    listMetrics.append(listM)
avgPrecision = totalPrecision / (Y_test.shape[0])
avgRecall = totalRecall / (Y_test.shape[0])
print("Average Precision")
print(avgPrecision)
print("Average Recall")
print(avgRecall)

Average Precision
0.524084098219
Average Recall
0.538675766847


In [174]:
#nclasses = 7.0
#nclasses1 = 7
 
#class_weights = {k:len(Y_train)/(nclasses*(Y_train[:,k]==1).sum()) for k in range(nclasses1)}

#print class_weights

#model.fit(X_new_train,Y_train, class_weight=class_weights, epochs=250,batch_size=8)